# Enunciado


Este trabalho usa SageMath nas suas implementações


1. Pretende-se construir em torno de uma cifra assimétrica um conjunto de técnicas criptográficas destinadas a fins distintos. Apesar de todas as alíneas do problema poderem ser  respondidas com a maioria das cifras assimétricas clássicas ou pós-quânticas, neste problema vamos exemplificar o processo com uma técnica simples da família Diffie-Hellman nomeadamente a cifra assimétrica ElGamal com parâmetros de segurança $\,\lambda\,$.
    1. Implemente um esquema  PKE $\,\mathsf{ElGamal}(\lambda)\,$ (ver Capítulo 4) num subgrupo de ordem prima $\,q\,$,  com $\,|q|\geq \lambda\,$, do grupo multiplicativo $\,\mathbb{F}^\ast_p\,$ com $\,p\,$ um primo que verifica $\,|p| \geq \lambda\times|\lambda|$ . Identifique o gerador de chaves e os algoritmos de cifra de decifra neste esquema. Identifique o núcleo deterministico do algoritmo de cifra.
    2. Supondo que a cifra que implementou é IND-CPA segura (de novo Capítulo 4), usando a transformação de Fujisaki-Okamoto implemente um PKE que seja IND-CCA seguro.
    3. A partir de (b) construa um esquema de KEM que seja IND-CCA seguro.
    4. A partir de (b) construa uma implementação de um protocolo autenticado de "Oblivious Transfer" $\,\kappa$-out-of-$n\,$.

# **Exercício 1.a.**

EL GAMAL
Qualquer PKE é determinado por três algoritmos: geração de chaves, cifra e decifra:

 ----------<br><br>$\text{GenKeys}(\lambda)$                                 …       $\lambda\;$ é o parâmetro de segurança<br><br><br>    - gerar aleatoriamente um primo $\,q \approx 2^\lambda$                                      <br>    - gerar um primo $p$  tal que  $\,\mathbb{F}_p^\ast\,$ tem um sub-grupo de ordem $\,q\,$ ; calcular um gerador $g$ desse sub-grupo<br>    - gerar aleatoriamente  $\,0 <s < q\,$ ,  a chave privada<br>    - calcular e  revelar  a chave pública   $\,\mathsf{pk} \equiv \langle p,q, g,g^s\rangle$<br>----------<br><br>$\text{Enc}(\mathsf{pk},m)$                                   …   a mensagem $m$ é um elemento de $\mathbb{F}_p^\ast$ <br><br><br>    - obter elementos públicos  $\,p,q,g,g^s \,\gets\,\mathsf{pk}$<br>    - gerar aleatoriamente  $\,0 <\omega < q$ <br>    - calcular  $\,\gamma \gets g^\omega\;$ e $\,\kappa \gets (g^s)^\omega\,$.<br>    - construir  o criptograma $\,\mathbf{c}\gets \langle\,\gamma\,,\, m\times\kappa\,\rangle\,$<br>----------<br><br>Note-se que se verifica $\,\kappa = \gamma^s\,$.<br> 
----------<br><br>$\text{Dec}(\mathsf{sk},\mathbf{c})$  …  $\mathsf{sk} = s$ é a chave privada<br><br>
- obter a chave privada $s$<br>    
- obter o criptograma $\mathbf{c} = \langle \gamma, \delta \rangle$<br>    
- calcular $\kappa \gets \gamma^s \mod p$<br>    
- calcular $\kappa^{-1} \mod p$<br>    
- recuperar a mensagem original: $m \gets \delta \times \kappa^{-1} \mod p$<br>    

In [1]:
print_tentativas_genKeys = False

In [2]:
from sage.all import *
lambda_security = 128  # Define um tamanho de bits para q

Para encontrar um gerador do grupo multiplicativo temos que encontrar um número cuja ordem seja igual à do grupo, i.e.:
- Dada a ordem do grupo multiplicativo $F_p^* = \phi(p)$ o gerador g tem de ter ordem $\phi(p)$, ou seja, $g^{\phi(n)}=1 \ mod \ p$

In [3]:
def find_generator(p):
    """Encontra um gerador do grupo multiplicativo F_p^*."""
    if not is_prime(p):
        raise Exception("O p de input não é primo")

    phi_p = p - 1  # Para p primo, phi(p) = p - 1

    fatoracao = factor(phi_p)
    fatores_primos = list(set([q for q, e in fatoracao]))
    
    #print(f"Fatoração de phi(p): {fatoracao}")
    #print(f"Fatores primos de phi(p): {fatores_primos}")

    # Itera sobre possíveis geradores
    for g in range(2, p):
        if gcd(g, p) != 1:
            continue  # Ignora números não coprimos com p

        is_gerador = True
        for q in fatores_primos:
            if pow(g, phi_p // q, p) == 1:
                is_gerador = False
                break

        if is_gerador:
            return g 

    return None  

In [4]:
def gen_keys(lambda_security):
    print("----------------------------------Geração de chaves:--------------------------------------")
    # Gerar aleatoriamente q com bit_length maior que lambda
    q = random_prime(2^129 - 1, False, 2^128)
    print("Parâmetro q gerado:",q)
    
    #Gera-se sucessivamente inteiros pi = q*2^i+1 até que pi seja um primo suficientemente grande, ou seja |p| > 1024
    i = 1
    p_i = q * (2^i) + 1 
    tamanho_desejado = lambda_security * lambda_security.nbits()
    
    while True:
        # Caso o p_i não convergir tenta um novo q
        if p_i.nbits() > 2500:
            q = random_prime(2^129 - 1, False, 2^128)
            i = 1
            p_i = q * (2^i) + 1
            print("p não convergiu, novo valor de q: ",q)
            
        if is_prime(p_i) and p_i.nbits() >= tamanho_desejado:
            break  
    
        i += 1
        p_i = q * (2^i) + 1
        if print_tentativas_genKeys:
            print(f"Tentativa {i}: p_i = {p_i} (Tamanho: {p_i.nbits()} bits)")

    p = p_i
    print("Parâmetro p gerado:",p)
    
    # Fp é um grupo multiplicativo se para todo o x pertencente a Fp gcd(x,p) = 1 (trivial já que p é primo)
    # Criar o corpo finito F_p
    F_p = GF(p)
    # O grupo multiplicativo F_p^* é o conjunto de elementos não nulos de F_p
    F_p_star = F_p.unit_group()
        
    # Agora obtemos um gerador do subgrupo de ordem q
    g = find_generator(p)
    print("gerador (g):",g)
    
    # Gerar aleatoriamente a chave privada 0<s<q
    s = randint(1, q-1)
    print("chave privada (s):", s)
    print("---------------------------------------------------------------------------------")
    return (p, q, g, pow(g, s, p)), s

## Escolha de p:
A ordem desse grupo é $\,n = p - 1\,$ e para que o DLP seja  complexo não basta apenas que $\,p\,$ seja grande: é também necessário que o maior factor primo de $\,(p-1)\,$ seja também grande. 
Para garantir estas condições o primo $\,p\,$  é gerado de uma determinada forma:

1. Gera-se um primo $\,q\,$ grande: com mais de $\lambda$ bits de tamanho; este vai ser o maior factor de $\,(p-1)\,$
2. Gera-se sucessivamente inteiros  $\,p_i\;=\;q\,2^i + 1\,$ até que $\,p_i\,$ seja  um primo suficientemente grande .

## Como é que $F_p^*$ tem um subgrupo de ordem q? (Teorema de Lagrange)
O Teorema de Lagrange diz que, se um grupo G tem ordem finita e H é um subgrupo de G, então |H| é um divisor de |G|.

Visto que $F_p^*$ tem ordem $p-1$ e $p-1$ divide $2q$ (e consequentemente $q$) então podemos afirmar que $F_p^*$ tem um subgrupo de ordem q. 

------------------

In [5]:
def enc(pk, m):
    p, q, g, g_s = pk
    omega = randint(1, q-1)  # Escolhe um valor aleatório ω entre 0 e q
    gamma = pow(g, omega, p)  # γ = g^ω mod p
    kappa = pow(g_s, omega, p)  # κ = (g^s)^ω mod p
    c = (gamma, (m * kappa) % p) 
    return c

------------------

In [6]:
def dec(sk, pk, c):
    p, _, _, _ = pk
    gamma, delta = c
    kappa = pow(gamma, sk, p)  # κ = γ^s mod p
    kappa_inv = inverse_mod(Integer(kappa), Integer(p))  # Calcula o inverso de κ mod p
    m = (delta * kappa_inv) % p  # Recupera a mensagem original
    return m

------------------

In [7]:
# Exemplo
pk, sk = gen_keys(lambda_security)  # Geração de chaves
m = randint(1, pk[0]-1)  # Mensagem aleatória em F_p*
c = enc(pk, m)  # Cifra a mensagem
m_dec = dec(sk, pk, c)  # Decifra a mensagem

# Exibir os resultados
print(f"Mensagem original: {m}")
print(f"Criptograma: {c}")
print(f"Mensagem decifrada: {m_dec}")
print(f"Decifração bem-sucedida? {m == m_dec}")
print("---------------------------------------------------------------------------------")

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 366612083898431147301155577919442505793
Parâmetro p gerado: 69107033191929826119404786459264957023992978522538375825938486719385472720010042049845355079174715448047150174127074633794182149117233409600417537167957765876773845578331493332977401449315466238714179965568521215343401473648315264139279010807008602086755418460078369040449173673849978875713231653430612822291297607019103881510887434409583582267087781889
gerador (g): 3
chave privada (s): 164075583970069760899753505327901761120
---------------------------------------------------------------------------------
Mensagem original: 525405339482536323235354782762872957774895663086293413473082283097303901299350695908744037381396016150488574482871207766579408098561274706137812953379717879421336666848101672280286370883049012794189950292376226298860909112397524237628515889801313435295980645674353137653293085446900204970783696192401

--------

# **Exercício 1.b.**

Supondo que a cifra que implementou é IND-CPA segura (de novo Capítulo 4), usando a transformação de Fujisaki-Okamoto implemente um PKE que seja IND-CCA seguro.

### **Transformar um  PKE-IND-CPA em um PKE-IND-CCA**

A transformação FO original constrói, a partir de $\,(E_p,D_s)\,$,  um novo esquema de cifra assimétrica $\,(E'_p,D'_s)\,$ , usando um  “hash” pseudo-aleatório $\,h\,$ de tamanho $\,\lambda\,$ e um “hash” pseudo-aleatório $\,g\,$ de tamanho $\,|x|\,$.

O algoritmo de cifra parametrizado pelos dois “hashs”  $\,h,g\,$    é 

  $$E'_{p}(x)\;\equiv\;\vartheta\,r\gets \{0,1\}^\lambda\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\vartheta\,c\gets f_p(r,r') \,\centerdot\, (y\,,\,c)$$

O algoritmo $\,D'_{s}\,$ rejeita o criptograma se detecta algum sinal de fraude. 


$$D'_{s}(y,c)\;\equiv\;\vartheta\,r \gets D_s(c)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\mathsf{if}\;\;c\neq f_p(r,r')\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$

### Definições de variáveis

In [8]:
print("----------------------------------Definição de veriáveis----------------------------------")
lambda_bits = 128

pk, sk = gen_keys(lambda_bits)
x = randint(1, pk[0]-1)  # Mensagem aleatória em F_p*
length_in_bytes = (x.bit_length() + 7) // 8
x = x.to_bytes(length_in_bytes, byteorder='big')
print("length x:",len(x))
print("Mensagem de input:",x)
print("-------------------------------------------------------------------------------------------")

----------------------------------Definição de veriáveis----------------------------------
----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 527631778565279328552715835647233433529
p não convergiu, novo valor de q:  579870760771153711949407231341541825681
Parâmetro p gerado: 32261636153106694249085209366157559522357107106660094142296181845294505425422415467886723091432525881290037743389087443376220566849501159849358337305475265449303968927990050411809089029251892702419124395079362418410243312712661308634049765299344297408049275933063203131042305877407014516850217325774359267363736554740501408426580866070776808620516804179399569570315555045377
gerador (g): 3
chave privada (s): 205857933387831295781825345216766073685
---------------------------------------------------------------------------------
length x: 156
Mensagem de input: b"\x01+\xaa\x91\xf2\xd1\x94\x8a\x89L|\x02\x85\xad\xcc\x97\xb4\xc2=R\xd95\xb1\xca\xecN\x92%\xc3\x17\

-------

### Funções auxiliares:
- $g$, um "hash" pseudo-aleatório de tamanho $|x|$
- $h$, um "hash" pseudo-aleatório de tamanho $lambda$
- $f_p$ o núcleo determinístico da cifra ElGamal

In [9]:
import hashlib

def g(r):
    """Hash pseudoaleatório g(r) com tamanho igual ao da mensagem x"""
    g = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    g.update(r_bytes)
    final_hash = g.digest()  # Truncar para o tamanho de x
    while len(final_hash) < len(x):
        g = hashlib.sha512()
        g.update(r_bytes)
        final_hash += g.digest()
    #print("g(r) hash:", final_hash[:len(x)])
    return final_hash[:len(x)]

def h(r, y):
    """Hash pseudoaleatório h(r, y) com tamanho lambda_bits"""
    h = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    ry = bytes(a ^^ b for a, b in zip(r_bytes, y))
    h.update(ry)
    full_hash = h.digest()[:lambda_bits // 8]  # Truncar para lambda bits
    #print("h(r, y) hash:", full_hash)
    return full_hash

#Núcleo determinístico da função enc anterior
def f_p(pk, r, rlinha):
    p, q, g, g_s = pk
    gamma = pow(g, rlinha, p)  # γ = g^ω mod p
    kappa = pow(g_s, rlinha, p)  # κ = (g^s)^ω mod p
    return (gamma, (r * kappa) % p)

### Cifra IND-CPA segura 

In [10]:
print("----------------------------------CIFRA----------------------------------")
def enc_fujisaki(pk,x):
    r = ZZ.random_element(2^(lambda_bits - 1), 2^lambda_bits)
    print("r:",r)
    y = bytes(a ^^ b for a, b in zip(x, g(r)))
    rlinha = h(r,y)
    c = f_p(pk,r,int.from_bytes(rlinha,"big"))
    return (y,c)

(y,c) = enc_fujisaki(pk,x)
print("(y,c) = ",(y,c))
print("-------------------------------------------------------------------------------------------")

----------------------------------CIFRA----------------------------------
r: 337362501341971141411018721255327700780
(y,c) =  (b';\xd9\x81\x08\x18\x03v\x9d\xe5{/\xfc\x99\xc9!\\\x10\xf2\x9f\xe1)\xb2\x8d\xe3\xa2s\xad\xf1\xa2\xb30\xa8T9"\x83e\x10\xd0=\xa3\xd4\x91r\x84\xe1\xfen\xc6\x16\xe7\xbe\xea\xe8\x8e}H\x18[s\xceu\x9e[\xbb\'\xddi\xac\xab\xc6\xe9\x12\xa8\xd1\x1f1\x95\xf9\x87\xbb\x00\xed[\xeeH\x82Tb\x95\xc0\x8e\x8a\xc4\x0cQ\xa0\xf3Z\x05x\x146\x82n\x8ff>\xc8\x0fN\xa6\xf8\xc5%\x1akfG\x80R\x90\n\x1c\xfa\xd7\x8f \xb6K\'\xb3;W\x98L\xc6\xc8\x0e`"\xff\x97\x9e\xd3\x7f\xe1\xc8-\xac\xef\xfd5\xe5\x8aZ', (17055642291148646609079067918370833638900181737407364212556277229258044203520297358441814513050982623953056986618903328553076185661416504989083259079025037120941825024829698066386088726370832565785128864075082813153052058999222031964102130017778175201260427821598908681250992549038217509379458740534742293380046114227688230031090977482841559470375580964068898773571744349715, 1203053521264487355288459

### Decifra

In [11]:
print("----------------------------------------DECIFRA--------------------------------------------")
def dec_fujisaki(sk,pk,y,c):
    r = dec(sk,pk,c)
    print("r:",r)
    rlinha = h(r,y)
    if c != f_p(pk,r,int.from_bytes(rlinha,"big")):
        raise Exception("O criptograma não corresponde a fp(r,r'), absurdo")
    else:
        res = bytes(a ^^ b for a, b in zip(y, g(r)))
        return res

print("Mensagem decifrada: ",dec_fujisaki(sk,pk,y,c))
print("x == dec_fujisaki(sk,pk,y,c)?: ",x == dec_fujisaki(sk,pk,y,c)) 
print("-------------------------------------------------------------------------------------------")

----------------------------------------DECIFRA--------------------------------------------
r: 337362501341971141411018721255327700780
Mensagem decifrada:  b"\x01+\xaa\x91\xf2\xd1\x94\x8a\x89L|\x02\x85\xad\xcc\x97\xb4\xc2=R\xd95\xb1\xca\xecN\x92%\xc3\x17\x1f8\x1d\xa1\x9aW\x1c\xb62\x97FGg'\xf4\xff\x89\xf5.u\xca\xc7\xfc&$q\x96=T\x1f}\x83\x7f\xe6\x81\xd5\xf6\xf0Fy$\xfe~\x9f\x82\xe1-\xf1\x14L\x1f0O\xe8\x1e\xcf\xbe},\xa8\xffZ\xeb`#\xc1\xe9k\xe2\xd1\x01\xb2\xd4(\x8b\x1c\x90k\xb8\x119=\x10\xa6\x08c}\xa8\xed\x8c\x8c\xb5\x05pI!n\x9d\x8c\xb9\x0c*\xd1\x85z[\xaa\xff]\x9e>\x9bzUwOC{\xdd+\xd3\xd4{\xd8\xb5\x8e"
r: 337362501341971141411018721255327700780
x == dec_fujisaki(sk,pk,y,c)?:  True
-------------------------------------------------------------------------------------------


# **Exercício 1.c.**

Associado a um mecanismo de encapsulamento de chaves existe um algoritmo de revelação ou KRev (“_key revelation_”), que a partir do encapsulamento de uma chave, revela-a.  Naturalmente, por analogia com as cifras assimétricas, o KEM é um algoritmo  público, total e probabilístico,  enquanto que o KRev é um algoritmo privado, parcial e determinístico.

A condição de correção será
                                            $$(e,k)\,\gets\,\mathsf{KEM}\quad\;\text{sse}\;\quad \mathsf{KRev}(e)\,\simeq\, k$$

Combinando o exercício anterior com um KEM o esquema pode-se descrever pelo par de algoritmos:

$$\left\{\begin{array}{lcl}E(x) & \equiv & \vartheta\,(e,k)\gets\mathsf{KEM}\,\centerdot\,(e\,,\,k \oplus x) \\ D(e,c) & \equiv & \vartheta\,k \gets \mathsf{KRev}(e)\,\centerdot\,k\oplus c\end{array}\right.$$

Substituindo KEM pela função de cifra do exercício (b) obtemos uma segurança IND-CCA para a encapsulação da chave.
Como o KEM é um mecanismo apenas de encapsulamento da chave iremos utilizar em conjunto com um DEM (neste caso OTP) para demonstrar que resulta para cifrar uma mensagem, como é demonstrado no diagrama:

![PKE standard KEM + OTP](KEMDEM.png)

----------

### Definições de variáveis

In [12]:
lambda_bits = 128

pk, sk = gen_keys(lambda_bits)
x = randint(1, pk[0]-1)  # Mensagem aleatória em F_p*
length_in_bytes = (x.bit_length() + 7) // 8
x = x.to_bytes(length_in_bytes, byteorder='big')

print("Mensagem aleatória em F_p*: ",x)

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 604884144537685470257801027021436073421
p não convergiu, novo valor de q:  562711663498646999761799762645931314671
Parâmetro p gerado: 178731226771262448024886227910045514755010740489419621880658143241379515832225587209336958662119740295704815409306393430219854943176109495769538888867504120721163014908698879046182536284666531124548762604813958694367134843977791653886278673782079766334208386359738955286051415524500894845554748528298726982815092518950486858571472590146704062963474439557409721044761235552692632654674247781186780852637136707491343630337
gerador (g): 3
chave privada (s): 380557587300284706791950697925045009951
---------------------------------------------------------------------------------
Mensagem aleatória em F_p*:  b'\x11C\xc5\x0b\x1d\x9b\x8b`\xae\x92\xd1FY-Uu\xa5L@\x04_Ud\x12?\x08\xe2\xf8(\xf2\xe9v\x87\xc6\xc81\xe7\x86\xca\xe5%Gc\x99\x86\xec\xeeB\xd5\x05\x1a\x

-------

### KEM + DEM Encrypt

Construir algoritmo de cifra tal que
$ E(x) \equiv \vartheta (e,k) ← KEM⋅(e,k⊕x)  $ 
onde 
- $k$ é gerado aleatoriamente;
- $e$ é o encapsulamento dessa chave.

In [13]:
def kem_encrypt(pk, x):
    # Gerar chave simétrica aleatória k
    k = ZZ.random_element(2^(8*len(x)-1), 2^(8*len(x))).to_bytes(len(x), 'big')
    print("k: ",k)
    # Utilizar cifra do ex1.b. como KEM para encapsular k
    e = enc_fujisaki(pk,k)
    
    # Aplicar DEM (neste caso OTP) da chave gerada k com a mensagem x
    ciphertext = dem(x,k)

    return e, ciphertext

def dem(x,k):
    ciphertext = bytes(a ^^ b for a, b in zip(x, k))
    while len(ciphertext) < len(x):
        ciphertext += bytes(a ^^ b for a, b in zip(x[len(ciphertext):], k))
        ciphertext[:len(x)]
    return ciphertext

-------

### KEM + DEM Decrypt

A decifração é dada por:
$ D(e,c) ≡ ϑk←KRev(e)⋅k⊕c $
onde 
- $KRev$ no nosso caso é a técnica de decifração utilizada no exercício 1.b.;
- $e$ é o encpasulamento da chave gerada;
- $c$ o criptograma recebido de KEM.

In [14]:
def kem_dem_decrypt(sk, pk, e, ciphertext):
    (y, c) = e
    k = dec_fujisaki(sk, pk, y, c)
    print("k: ",k)
    x = dem(ciphertext,k)
    return x

---------

### Teste

In [15]:
print("-------------------------------------------------------------------------------------------------------------------")
print("mensagem (x): ",x)
e, ciphertext = kem_encrypt(pk,x)
print("-------------------------------------------------------------------------------------------------------------------")
print("e: ",e)
print("-------------------------------------------------------------------------------------------------------------------")
print("k (+) x: ",ciphertext)
print("-------------------------------------------------------------------------------------------------------------------")

mensagem = kem_dem_decrypt(sk, pk, e, ciphertext)

print("mensagem (final): ",mensagem)
print("-------------------------------------------------------------------------------------------------------------------")
print("mensagem inicial == mensagem final ?: ",x == mensagem)
print("-------------------------------------------------------------------------------------------------------------------")

-------------------------------------------------------------------------------------------------------------------
mensagem (x):  b'\x11C\xc5\x0b\x1d\x9b\x8b`\xae\x92\xd1FY-Uu\xa5L@\x04_Ud\x12?\x08\xe2\xf8(\xf2\xe9v\x87\xc6\xc81\xe7\x86\xca\xe5%Gc\x99\x86\xec\xeeB\xd5\x05\x1a\xe7\xc65\x06\xf2\x8b\xca\x93\xc4(\xb62E\xe4\xcc\xc7\x9d\xe3\x9dJK\x0e\xa8\x97Gxw\xd4\xd1\n\\\xa0r.\x100\t\xed\x1d\x92\x0f\xa9\\<I\xc3,\xb1/\x93S>+\x92\xfe\x83\xf0WQG_z\xc6\xa0\xabw\xbb\r|\xc3\x02\xfe&\x14s%/\xddLZ\x8c\xb8\xe1\xfb\x1d JXH\x1d\xce\x13\xbf\xaa\x0f\x8f\xb3\x94*\\\x98\x94\x83}\x8f\x00\x06J\xb5L\x18\xe1\xb1F.\x84w(\x0c\xd2\xac2\xf9'
k:  b'\xb9\x8f\xcf\xa0\xa5\xff\xc8\xf0\xc6\xc5\xa3SY=&Tn\xe6\x06\x9e\xe0l\xc7<I\xda\x9b\x10\xbaz\xc9\xe8\xa4\x846+5k\xd4\x8e\xd2\xa0\x98\x96\xbeA\x04B\xbefC\x0f]"\x16L?\xd1\xc6wt\x0f\xa6\xf8wl}\xda\x97\xbe\xfa\xddL*\xa0\xc3\x14l\x8dR\x11\x02\xbe\xd9\x0e\x06\x10\x8e\x90\x15/-\xa9\x14\x99U\x0b\x19e\xb8\x05\xd9\x0b\t\xa8&\xe9?\xe3\x81\xe4\xade\xc7\xa6\xb5\xa4\xe8_\x8aj\xb2%\\\

----------------

# Exercício 1.d.


O protocolo de “oblivious transfer”  implementa um mecanismo de transferência de informação entre dois agentes: o **Provider** (também designado por Sender) e o **Receiver** (também designado por Adversário) . Em linhas gerais, o protocolo caracteriza-se da forma seguinte:


1. O **Provider** põe à disposição para comunicação futura $\,n\,$ items de informação (ou mensagens) que ele enumera como $\,m_1, m_2, \cdots,m_{n}\;$ e que armazena de forma privada. 
    Nesta fase a única informação tornada pública é o número  de mensagens  $\,n$ .
2. O Receiver informa o **Provider** que pretende receber $\,\kappa\,$ das $\,n\,$ mensagens
3. Caso o **Provider** aceite o par $(n,\kappa)\;$ os dois agentes, a começar pelo **Provider**, trocam uma sequência de mensagens e, no final,
    1. O **Receiver** passa a conhecer exatamente  $\,\kappa\,$ mensagens mas continua a ignorar o conteúdo de todas as restantes $\,n-\kappa\,$ mensagens.
    2. O **Provider** ignora a identificação (“is oblivious of”)  das $\,\kappa\,$ mensagens que o Receiver passou a conhecer.

O protocolo usa um esquema PKE   $\,\{(E_p,D_s)\}_{(s,p)\in\mathcal{G}}$ que neste caso irão ser a cifra e decifra do exerício 1.b.. 

----

### Criterion

O critério $C{κ,n}$ define quais vetores de chaves públicas **p** são considerados válidos. O **Receiver** deve criar um vetor **p** onde:

- Algumas entradas são chaves públicas "boas" (as que correspondem às mensagens que quer receber).
- O resto das entradas são chaves públicas "más" (geradas para manter a segurança do protocolo).

O critério $\,\mathcal{C}_\kappa\,$ depende da estrutura algébrica usada pelo mecanismo de geração de pares de chaves. Vamos considerar:

- Cada chave pública, válida ou não, é codificada por um inteiro em $\,\mathbb{F}_p^*\,$.
- Uma matriz de “rank” completo  $\,\mathsf{A} \in \mathbb{F}_p^{* n\times(n-\kappa)}\,$ e um vector $\,\mathsf{u}\neq 0\in \mathbb{Z}_q^{n-\kappa}\,$ ; estes elementos são gerados por um XOF a partir de uma “seed”   $\,\rho\,$ . A  “seed” é aleatoriamente gerada e os restantes elementos são construídos com o XOF, por tentativas, até se verificarem as condições exigidas.
- O critério tem a forma de um sistema de equações lineares  $\;\mathsf{p} \times \mathsf{A}\,=\,\mathsf{u}$ .

In [16]:
import numpy as np

class CknCriterion:
    def __init__(self, kappa, n, q):
        self.kappa = kappa
        self.n = n
        self.q = int(q) 
        self.seed = np.random.randint(0, 2**32)
        self.A = self.generate_A()
        self.u = self.generate_u()
        self.Fp = GF(q).unit_group()
    
    def generate_A(self):
        """Gera a matriz A usando XOF a partir da seed"""
        np.random.seed(self.seed)
        A = random_matrix(GF(self.q), self.n, self.n - self.kappa)
        return A
    
    def generate_u(self):
        """Gera o vetor u, que deve ser não nulo"""
        np.random.seed(self.seed + 1) 
        u = vector(GF(self.q), [randint(1, self.q - 1) for _ in range(self.n - self.kappa)])
        return u
    
    def verify(self, p):
        """Verifica se p satisfaz o critério Ckn, ou seja, se p * A = u"""
        if len(p) != self.n:
            raise ValueError(f"p deve ter {self.n} elementos")
        
        p_values = [x[0] if isinstance(x, tuple) else x for x in p]
        print("p_values:",p_values)
        
        # Converter p_values para um vetor no corpo finito Z_q
        Zq = GF(self.q)
        p_vector = vector(Zq, p_values)
    
        # Calcular p * A no corpo finito Z_q
        A_matrix = matrix(Zq, self.A) 
        pA = p_vector * A_matrix 
    
        # Verificar se pA é igual a u
        u_vector = vector(Zq, self.u)  # Converter u para um vetor no corpo finito Z_q
        return pA == u_vector
    
    def print_criterion(self):
        print(f"Matriz A:\n{self.A}")
        print(f"Vetor u:\n{self.u}")

## Provider

In [17]:
class Provider:
    def __init__(self, pk, sk, n_mensagens):
        self.pk,self.sk = gen_keys(lambda_bits) #(p, q, g, pow(g, s, p)) , s
        self.numero_de_mensagens = n_mensagens
        # Informação privada das mensagens:
        self.messages = [f"mensagem{i}" for i in range(n_mensagens)]
        self.criterion = None
    def define_criterion(self,kappa):
        n = self.numero_de_mensagens
        q = pk[1]
        self.criterion = CknCriterion(kappa, n, q)

Inicialmente:
- Gera chaves
- Gera mensagens
- Expõe apenas o número de mensagens, matriz $A$ e vetor $u$

In [18]:
n_mensagens = 100
provider = Provider(None,None,n_mensagens)

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 478059461990121761147462853540001071043
Parâmetro p gerado: 8429011993812721819844378270294614395744643039980929259044475094639151971478827918984340673996832005792234705881000058554435691619214062059734766037438912236961872841490144166005446902469564135392049705356831836108126702820756031589575396314743381236887383170951132903976178548812131991006525593101917170181174628278013766952369636788744049009821542643567148255244457694561552194798550937539064765612033
gerador (g): 3
chave privada (s): 270220245422073796200357473228367340635
---------------------------------------------------------------------------------


## Receiver

In [19]:
import hashlib
from random import sample

class Receiver:
    def __init__(self, k, n_mensagens, q):
        self.k = k  # Número de chaves privadas geradas
        self.n_mensagens = n_mensagens
        self.q = q 
        self.I = sample(range(n_mensagens), k)  # Seleção aleatória de k índices
        self.e = self.enumeration(self.I)  
        self.p,self.s_values = self.generate_keys()  
        self.s = self.generate_secret()
        self.tau = self.generate_authentication_tag()

    def enumeration(self, I):
        """Cria a função de enumeração que mapeia {1, 2, ..., κ} para os elementos de I ordenados"""
        I_sorted = sorted(I)
        return {i + 1: I_sorted[i] for i in range(len(I_sorted))}

    def generate_secret(self):
        """Gera um segredo aleatório (simulado como um número grande)"""
        return ZZ.random_element(2**32)

    def generate_keys(self):
        """Gera κ chaves privadas e publicas usando o genkeys de ElGamal"""
        vetor_pk = [0] * self.n_mensagens 
        vetor_sk = []
        for i in range(1,self.k+1):
            pk,sk = gen_keys(lambda_bits) # (p, q, g, pow(g, s, p)), s
            vetor_sk.append(sk)
            vetor_pk[self.e[i]] = pk
        return vetor_pk,vetor_sk

    def generate_authentication_tag(self):
        """Gera a tag de autenticação hash(I, s)"""
        data = str(self.I) + str(self.s)
        return hashlib.sha256(data.encode()).digest()

    def complete_p_vector(self, A, u):
        """Completa o vetor p para satisfazer p * A = u no corpo finito Z_q"""
        Zq = GF(self.q)  # Define o corpo finito Z_q
    
        # Identificar os índices já preenchidos (valores diferentes de 0)
        filled_indices = [i for i in range(self.n_mensagens) if self.p[i] != 0]
        filled_values = vector(Zq, [self.p[i][0] if isinstance(self.p[i], tuple) else self.p[i] for i in filled_indices])
    
        # Criar a matriz A_filled (linhas correspondentes aos índices preenchidos)
        A_filled = matrix(Zq, [A[i] for i in filled_indices])
    
        # Criar a matriz A_empty (linhas correspondentes aos índices vazios, ou seja, onde p[i] == 0)
        A_empty = matrix(Zq, [A[i] for i in range(A.nrows()) if i not in filled_indices])
    
        # Calcular u' = u - (filled_values * A_filled)
        u_prime = vector(Zq, u) - filled_values * A_filled
    
        # Resolver o sistema linear A_empty^T * p_empty = u' no corpo finito Z_q
        try:
            A_empty_T = A_empty.transpose()
            p_empty = A_empty_T.solve_right(u_prime)
        except:
            # Se o sistema for singular, tentar solução alternativa (ex: mínimos quadrados)
            p_empty = A_empty_T.pseudoinverse() * u_prime
    
        # Preencher os elementos desconhecidos no vetor p (apenas onde p[i] == 0)
        empty_indices = [i for i in range(self.n_mensagens) if self.p[i] == 0]
        for i, idx in enumerate(empty_indices):
            # Gerar a chave "má"
            p_mau, q_mau, g_mau, gs_mau = self.gen_mau_keys() 
            self.p[idx] = (p_mau, q_mau, g_mau, gs_mau)
        return self.p
    
    def gen_mau_keys(self, max_attempts=100):
        """Gera uma chave 'má' no formato (p, q, g, g^s)"""
        attempts = 0
        while attempts < max_attempts:
            # Gerar q_mau como um número primo aleatório
            q_mau = random_prime(2^128, False, 2^127)
    
            # Tentar encontrar um k tal que p_mau = q_mau * 2^k + 1 seja primo
            for k in range(1, 10):  # Limitar k a um intervalo razoável
                p_mau = q_mau * (2^k) + 1
                if is_prime(p_mau):
                    # Criar o corpo finito F_p_mau
                    F_p_mau = GF(p_mau)
    
                    # Encontrar um gerador g_mau do grupo multiplicativo F_p_mau^*
                    g_mau = F_p_mau.multiplicative_generator()
    
                    # Gerar a chave privada s_mau aleatoriamente
                    s_mau = randint(1, q_mau - 1)
    
                    # Calcular g^s mod p
                    gs_mau = pow(g_mau, s_mau, p_mau)
    
                    return p_mau, q_mau, g_mau, gs_mau
    
            attempts += 1
    
        raise ValueError(f"Não foi possível gerar p_mau após {max_attempts} tentativas.")
    
    def print_info(self):
        print("----------------------------------------------------------------------")
        print(f"Seleção I: {self.I}")
        print("----------------------------------------------------------------------")
        print(f"Função de enumeração e: {self.e}")
        print("----------------------------------------------------------------------")
        print(f"Segredo s: {self.s}")
        print("----------------------------------------------------------------------")
        print(f"Chaves privadas s_i: {self.s_values}")
        print("----------------------------------------------------------------------")
        print(f"Vetor p (com chaves públicas mapeadas): {self.p}")
        print("----------------------------------------------------------------------")
        print(f"Tag de autenticação τ: {self.tau}")
        print("----------------------------------------------------------------------")

Escolhe o número de mensagens que pretende receber do Provider tendo acesso ao número de mensagens disponíveis

In [20]:
k = 20
receiver = Receiver(k,n_mensagens,provider.pk[1]) # Receiver escolhe o conjunto I já na sua construção

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 486442810527030314103382405881541889911
p não convergiu, novo valor de q:  377569782642478873907115701018881956299
Parâmetro p gerado: 49600048414471097261822416922941709710968385036689142127186371308625012763030106433463560494490394217794012819828050985298094917622464124093216190509982762918043910596761797841474527544366974021861805002403181081849461069109784516979236049924076756744865654139877406610818536586265001914622608795694212102388206321978039054969621984652801837618523888306791023483612911840784961746354376898772993
gerador (g): 3
chave privada (s): 226885061370998964098354702329544267932
---------------------------------------------------------------------------------
----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 377442959734070604324618431942065683347
Parâmetro p gerado: 6940883303076822657853461559025

### 1. O **Provider** gera o critério C_{k,n} e envia-o ao **Receiver**

In [21]:
provider.define_criterion(k)

### 2. O Receiver  escolhe um conjunto $\,I \subset \{1,n\}\,$ , de tamanho $\,\#I = \kappa\,$ , que identifica os índices das mensagens que pretende recolher.  
Seja $\;e\;$ a enumeração de $\,I\,$:  a função crescente $\,e\colon\{1,\kappa\}\to \{1,n\}\;$  cuja imagem é $\,I\,$.

O Receiver compromete-se com a escolha de mensagens da seguinte forma (dado o conjunto $I$ e a função crescente $e$):

1. Gera aleatoriamente um segredo $\,\mathbf{s}\,$ e , usando um XOF com$\,\mathbf{s}\,$ como “seed”, constrói$\,\kappa\,$  chaves privadas$\,s_1,\cdots,s_\kappa\,$ .
2. Para cada$\,i\in \{1,\kappa\}\,$ gera chaves públicas $\;\upsilon_i \gets \mathsf{pk}(s_i)\,$ e atribui o valor$\,\upsilon_i\,$ à componente de ordem$\,e(i)\,$ do vector $\,\mathsf{p}\;$; ou seja , executa$\,$ $\,\mathsf{p}_{e(i)} \gets \upsilon_i\,$
3. Gera uma “tag” de autenticação$\,$para a seleção$\,I\,$ e o segredo$\,\mathbf{s}\,$

$$\,\tau \gets \mathsf{hash}(I\,,\,\mathbf{s})\,$$

(Definido na classe:)

In [22]:
receiver.print_info()

----------------------------------------------------------------------
Seleção I: [79, 24, 3, 88, 96, 22, 97, 43, 57, 56, 23, 33, 0, 6, 53, 51, 28, 78, 8, 72]
----------------------------------------------------------------------
Função de enumeração e: {1: 0, 2: 3, 3: 6, 4: 8, 5: 22, 6: 23, 7: 24, 8: 28, 9: 33, 10: 43, 11: 51, 12: 53, 13: 56, 14: 57, 15: 72, 16: 78, 17: 79, 18: 88, 19: 96, 20: 97}
----------------------------------------------------------------------
Segredo s: 1770542929
----------------------------------------------------------------------
Chaves privadas s_i: [226885061370998964098354702329544267932, 178712025332514114829515272993095154187, 527408834047236640450652190479827982799, 436595198523929514097165485880435276671, 186783303180366133592672145674504211794, 140537400477684584179199104751971254909, 268711264571099086292722113046041843877, 178390265693830742827090273037613946003, 309800912060098770858160714005163176126, 106531227040635437936858043356432249503, 36

Em seguida completa a definição de $\,\mathsf{p}\,$ atribuindo às compompentes $\,\{\mathsf{p}_j\}_{j\in\!\!\!/I}$valores tais que o vetor de chaves públicas  $\,\mathsf{p}\,$ seja aceite pelo critério $\,\mathcal{C}_{\kappa,n}\,$.

Para que o vetor **p** satisfaça a equação $p×A=u$, onde **A** é a matriz gerada pelo critério $C_{k,n}$ e **u** é o vetor correspondente, o Receiver precisa preencher os espaços em **p** que não foram preenchidos pelas chaves públicas de forma que a equação seja satisfeita.

Mais concretamente essa forma de completar o vetor é descrita na classe Receiver:

O código realiza a seguinte sequência de operações para completar o vetor p de forma que satisfaça a equação p×A=up×A=u no corpo finito $Z_q$:

- Definição do corpo finito $Z_q$ usando ```GF(self.q)```.

- Identificação dos índices preenchidos no vetor $p$ usando ```filled_indices```, que são obtidos a partir dos valores em ```self.e```. Os valores correspondentes a esses índices são armazenados em ```filled_values```.

- Criação das matriz ```A_filled```  partir das linhas da matriz $A$ correspondentes aos índices preenchidos.

- Criação das matriz ```A_empty``` a partir das linhas da matriz $A$ correspondentes aos índices vazios.

- Cálculo do vetor $u′$ subtraindo a contribuição dos elementos preenchidos de $u$, ou seja, ```u′ = u − (filled_values × A_filled)```.

- Resolução do sistema linear $p_{empty} = A_{empty}^T . u′$ para preencher os valores desconhecidos de $p$ (chaves "más").

- Preenchimento de p, colocando os valores de $p_{empty}$ índices correspondentes no vetor p.

Finalmente o Receiver envia ao Provider a “tag” $\,\tau\,$ e o vetor $\,\mathsf{p}\,$

In [23]:
completed_p = receiver.complete_p_vector(provider.criterion.A, provider.criterion.u)
tau = receiver.tau

### 3. O Provider  determina  $\,\mathcal{C}_{\kappa,n}(\mathsf{p})\,$; se $\,\mathsf{p}\,$ não for aceite pelo critério então aborta o protocolo.

Se $\,\mathsf{p}\,$  for aceite,  então usa  a variante IND-CCA  da cifra IND-CPA com a “tag”  $\tau$

$$E'_{p}(x,\tau)\;\equiv\;\vartheta\,r\gets \{0,1\}^\lambda\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,r'\gets h(r,y,\tau)\,\centerdot\,\vartheta\,c\gets f_p(r,r') \,\centerdot\, (y\,,\,c)$$

cuja característica específica é o facto de se incluir o “tag” $\,\tau\,$ no “hash”  $\;h(r,y,\tau)\;$usado para construir a pseudo-aleatoriedade $\,r'\,$.

Usando esta cifra o Provider  constrói $\,n\,$ criptogramas 
$$\;(y_i,c_i) \gets E'_{p_i}(m_i)\quad$$  com $\,i\in\{1,n\}$
que envia para o Receiver.

Iremos definir um novo "hash" pseudo-aleatório $h$ para que consiga receber como argumento a tag $\tau$:

In [24]:
def h_OT(r,y,t):
    """Hash pseudoaleatório h(r, y, t) com tamanho lambda_bits"""
    h = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    ry = bytes(a ^^ b for a, b in zip(r_bytes, y))
    ryt = bytes(a ^^ b for a, b in zip(ry, t))
    h.update(ryt)
    full_hash = h.digest()[:lambda_bits // 8]  # Truncar para lambda bits
    #print("h(r, y) hash:", full_hash)
    return full_hash

E definir a cifra, que na realidade irá ser a do exercício 1.b. mas com um argumento extra $\tau$:

In [25]:
def f_p_OT(pk, r, rlinha):
    p, q, g, g_s = pk
    # Verificar se p é primo
    if not is_prime(p):
        raise ValueError(f"p = {p} não é um número primo. Não é possível criar o corpo finito GF(p).")


    # Calcular gamma e kappa
    gamma = pow(g, rlinha, p)  # γ = g^ω mod p
    kappa = pow(g_s, rlinha, p)  # κ = (g^s)^ω mod p

    # Multiplicar r_fp por kappa no corpo finito F_p
    result = (r * kappa) % p

    return (gamma, result)

In [26]:
def enc_fujisaki_OT(pk,x,t):
    # r gerado aleatoriamente com lambda bits
    r = ZZ.random_element(2^(lambda_bits - 1), 2^lambda_bits)

    # y = x XOR g(r)
    y = bytes(a ^^ b for a, b in zip(x, g(r)))

    # r' = h(r,y,tau)
    rlinha = h_OT(r,y,t)
    print(rlinha)

    # c gerado a partir do núcleo determinístico 
    c = f_p_OT(pk,r,int.from_bytes(rlinha,"big"))

    #Devolver criptograma
    return (y,c)

Determinamos então $C_{k,n}(p)$, se este for aceite ciframos todas as mensagens com as chaves públicas fornecidas em $p$:

In [29]:
import sys 
if not provider.criterion.verify(completed_p):
    print("------------------------------------------------------------------------")
    print("O vetor p foi aceite")
    print("p final:[ ")
    for key in completed_p:
        print(key)
    print("]")
    print("------------------------------------------------------------------------")
    ciphertext_vector = []
    i = 0
    for public_key in completed_p:
        if isinstance(public_key, tuple):
            byte_length = (sys.getsizeof(provider.messages[i]))
            ciphertext_vector.append(enc_fujisaki_OT(public_key, bytes(provider.messages[i], 'utf-8'), tau))
            i += 1
        else:
            raise ValueError(f"encontrada chave pública não primo: {public_key}")
else:
    print("O vetor p não foi aceite. Abortado")

p_values: [49600048414471097261822416922941709710968385036689142127186371308625012763030106433463560494490394217794012819828050985298094917622464124093216190509982762918043910596761797841474527544366974021861805002403181081849461069109784516979236049924076756744865654139877406610818536586265001914622608795694212102388206321978039054969621984652801837618523888306791023483612911840784961746354376898772993, 3738764474523369354855148753009499208497, 43951977694132208255731882404579945047297, 6940883303076822657853461559025834650333478542855730639208245340184580867919480917541941832065626881363819252449087732927940298447682925423182069251837101025575458732843780588017753438592659136751768897253741609608728429814503568611106881283129665934356692416548577117261780443604749176215955716507568111594294569047300342428267740659728295390044829284776691627181040300540385691496443042155630550773276887401778360579092531593019342991351891295144331203382014873395458720218739546013272098471937, 323925567

### 4. O Receiver  usa  a variante IND-CCA  da cifra IND-CPA com a “tag” de autenticação $\tau$
$$D'_{s}(y,c,\tau)\;\equiv\;\vartheta\,r \gets D_s(c)\,\centerdot\,\vartheta\,r'\gets h(r,y,\tau)\,\centerdot\,\mathsf{if}\;\;c\neq f_p(r,r')\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$
         
uma vez mais a única característica particular deste algoritmo é o uso da “tag” de autenticação $\,\tau\,$ na construção da pseudo-aleatoriedade $\;r'\gets h(r,y,\tau)\;$.


O agente Receiver 
   - conhece, porque criou,  a “tag” $\,\tau\,$que autentica o conjunto de mensagens escolhidas $\,I\,$ e  o respetivo conjunto de chaves públicas (as “boas” chaves).
    - conhece , porque gerou e armazenou num passo anterior,  as chaves privadas $\,s_i\,$ para todos $\,i\in I\,$  
    - conhece, porque recebeu do Receiver,  todos os criptogramas $\{(y_i,c_i)\}_{i\in\{1,n\}}\,$
        
Então,  para todo $\,i\in I\,$,  pode recuperar  a mensagem
                                                        $$m_i \,\gets\, D_{s_i}(y_i,c_i,\tau)$$ 


Como na cifra vamos fazer com que a decifra possa receber a tag de autenticação:

In [30]:
def dec_fujisaki_OT(sk,pk,y,c,tau):
    r = int(dec(sk,pk,c))
    rlinha = h_OT(r,y,tau)
    if c != f_p_OT(pk,r,int.from_bytes(rlinha,"big")):
        print("má decifração")
        return None
    else:
        print("boa decifração")
        res = bytes(a ^^ b for a, b in zip(y, g(r)))
        return res

In [37]:
# Decifrar e verificar as mensagens
decrypted_messages = []
for msg_number, idx in receiver.e.items(): 
    if idx < len(ciphertext_vector):  # Verifica se o índice é válido
        print(f"\nNúmero da mensagem: {msg_number}")
        print(f"Índice no vetor ciphertext_vector: {idx}")
        print(f"Chave privada (sk): {receiver.s_values[msg_number - 1]}")  # Ajuste para índice base 0
        print(f"Chave pública (pk): {receiver.p[idx]}")
        print("len(ciphertext_vector): ",len(ciphertext_vector))
        # Verifica se a mensagem foi cifrada corretamente
        if ciphertext_vector[idx] is None:
            print("idx:",idx)
            print(f"Mensagem {idx} não foi cifrada corretamente.")
            continue

        y, c = ciphertext_vector[idx]
        sk = receiver.s_values[msg_number - 1] 
        decrypted_message = dec_fujisaki_OT(sk, receiver.p[idx], y, c, tau)

        if decrypted_message is not None:
            print(f"Mensagem decifrada: {decrypted_message}")

            # Compara com a mensagem original no Provider
            if decrypted_int == provider.messages[idx].encode('utf-8'):
                print("provider.messages[idx]",provider.messages[idx])
                print("Decifração bem-sucedida! A mensagem decifrada corresponde à original.")
                decrypted_messages.append(decrypted_message)
            else:
                print("provider.messages[idx]",provider.messages[idx].encode('utf-8'))
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                print("Erro na decifração: A mensagem decifrada NÃO corresponde à original.")
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        else:
            print("Erro na decifração intencional.")


Número da mensagem: 1
Índice no vetor ciphertext_vector: 0
Chave privada (sk): 226885061370998964098354702329544267932
Chave pública (pk): (49600048414471097261822416922941709710968385036689142127186371308625012763030106433463560494490394217794012819828050985298094917622464124093216190509982762918043910596761797841474527544366974021861805002403181081849461069109784516979236049924076756744865654139877406610818536586265001914622608795694212102388206321978039054969621984652801837618523888306791023483612911840784961746354376898772993, 377569782642478873907115701018881956299, 3, 22717723164065415179675647333068428552736728466935953753961080149647107049257938104715801096803056847023197820208371194521221646267183153887146786561507506674452226398783337374093469292866287164954908375943749225693210288276079985660611734480716486603291587094413737255354647811769648888565881878403487840426660853781011260381687362282012164342898686739397614372262404052669384198986584646599973)
len(ciphertext_vector